In [7]:
import os
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from pathlib import Path
from PIL import Image

In [8]:
# config取得
base_dir = Path.cwd().parent.parent
config_path = base_dir / "config"
sys.path.append(str(config_path))

from config import IMAGES, GRID_SIZE, IMAGE_WIDTH, IMAGE_HEIGHT, WHITE_PIXEL_THRESHOLD, WHITE_RATIO_THRESHOLD
from config import GSD, BASE_COORDS

In [9]:
base_dir = Path(os.getcwd())
common_path = base_dir.parent.parent / "assets" / "images" 
path = common_path / "row_images"
output_path = common_path / "grid_images"
alpha_dir = common_path / "alpha"
cols = IMAGE_WIDTH // GRID_SIZE
rows = IMAGE_HEIGHT // GRID_SIZE

# フォント設定
try:
    font = ImageFont.truetype("arial.ttf", 32)
except:
    font = ImageFont.load_default()

# ラベルを生成するヘルパー関数
def get_excel_column_label(n):
    result = ""
    while n >= 0:
        result = chr(n % 26 + ord('A')) + result
        n = n // 26 - 1
    return result

# グリッド付き画像にセル名を描画
for img_name in IMAGES:
    img_path = path / img_name
    if img_path.exists():
        img = Image.open(img_path).convert("RGB")
        draw = ImageDraw.Draw(img)
        width, height = img.size
        cols = width // GRID_SIZE
        rows = height // GRID_SIZE
        for x in range(0, width, GRID_SIZE):
            draw.line([(x, 0), (x, height)], fill="red", width=2)
        for y in range(0, height, GRID_SIZE):
            draw.line([(0, y), (width, y)], fill="red", width=2)
        for row in range(rows):
            for col in range(cols):
                col_label = get_excel_column_label(col)
                row_label = str(row + 1)
                cell_label = f"{col_label}{row_label}"
                x = col * GRID_SIZE + 5
                y = row * GRID_SIZE + 5
                draw.text((x, y), cell_label, fill="blue", font=font)

        print(f"画像：{img_name}")
        display_img = img.resize((width // 1, height // 1))
        # display(display_img)
        output_file = output_path / img_name.replace(".jpg", "_grid.jpg")
        img.save(output_file)
    else:
        print(f"画像が見つかりません: {img_name}")

画像：八王子霊園.jpg
画像：小山内裏公園.jpg
画像：小山田緑地.jpg
画像：平山城址公園.jpg
画像：桜ヶ丘公園.jpg
画像：長沼公園.jpg


In [10]:
# records = []
# for mask_path in sorted(alpha_dir.glob("*.jpg")):
#     img_name = mask_path.name
#     park_name = img_name.replace(".jpg", "")
#     print(f"▶ 処理中: {park_name}")
#     try:
#         mask_pil = Image.open(mask_path).convert("L")
#         mask = np.array(mask_pil)
#         rows = mask.shape[0] // GRID_SIZE
#         cols = mask.shape[1] // GRID_SIZE
#         for row in range(rows):
#             for col in range(cols):
#                 y0, y1 = row * GRID_SIZE, (row + 1) * GRID_SIZE
#                 x0, x1 = col * GRID_SIZE, (col + 1) * GRID_SIZE
#                 tile = mask[y0:y1, x0:x1]
#                 white_ratio = np.sum(tile > WHITE_PIXEL_THRESHOLD) / (GRID_SIZE * GRID_SIZE)
#                 if white_ratio > WHITE_RATIO_THRESHOLD:
#                     cell_label = f"{get_excel_column_label(col)}{row+1}"
#                     records.append({
#                         "公園名": park_name,
#                         "セルラベル": cell_label,
#                         # "白比率": round(white_ratio, 3)
#                     })
#     except Exception as e:
#         print(f"エラー: {img_name} → {e}")

# df = pd.DataFrame(records)
# csv_path = base_dir / "ナラ枯れ主要被害公園エリア_抽出後.csv"
# df.to_csv(csv_path, index=False, encoding="utf-8-sig")
# print(f"CSV出力完了: {csv_path}")

In [11]:
# # 初期化
# records = []
# for img_name in IMAGES:
#     park_name = img_name.replace(".jpg", "") 

#     for row in range(rows):
#         for col in range(cols):
#             col_label = get_excel_column_label(col)
#             row_label = str(row + 1)
#             cell_label = f"{col_label}{row_label}"

#             records.append({
#                 "公園名": park_name,
#                 "セルラベル": cell_label
#             })

# df = pd.DataFrame(records)
# csv_path = base_dir / "ナラ枯れ主要被害公園エリア_抽出前.csv"
# df.to_csv(csv_path, index=False, encoding="utf-8-sig")
# print(f"CSV出力完了：{csv_path}")

In [12]:
# 緯度経度への変換関数
def pixel_to_latlon(base_lat, base_lon, pixel_x, pixel_y, gsd=0.2):
    dx_m = pixel_x * gsd
    dy_m = pixel_y * gsd
    delta_lat = -dy_m / 111000
    delta_lon = dx_m / (111000 * np.cos(np.deg2rad(base_lat)))
    return base_lat + delta_lat, base_lon + delta_lon

# 初期化
records = []
for img_name in IMAGES:
    park_name = img_name.replace(".jpg", "")
    mask_path = alpha_dir / img_name
    print(f"▶ 処理中: {park_name}")
    if not mask_path.exists():
        print(f"マスク画像が存在しません: {mask_path}")
        continue
    try:
        base_lat, base_lon = BASE_COORDS[park_name]
        mask_pil = Image.open(mask_path).convert("L")
        mask = np.array(mask_pil)
        rows = mask.shape[0] // GRID_SIZE
        cols = mask.shape[1] // GRID_SIZE
        for row in range(rows):
            for col in range(cols):
                y0, y1 = row * GRID_SIZE, (row + 1) * GRID_SIZE
                x0, x1 = col * GRID_SIZE, (col + 1) * GRID_SIZE
                tile = mask[y0:y1, x0:x1]
                white_ratio = np.sum(tile > WHITE_PIXEL_THRESHOLD) / (GRID_SIZE * GRID_SIZE)
                if white_ratio > WHITE_RATIO_THRESHOLD:
                    label = f"{get_excel_column_label(col)}{row + 1}"
                    px, py = col * GRID_SIZE, row * GRID_SIZE
                    lat, lon = pixel_to_latlon(base_lat, base_lon, px, py, gsd=GSD)
                    records.append({
                        "公園名": park_name,
                        "セルラベル": label,
                        "緯度": f"{lat:.5f}",
                        "経度": f"{lon:.5f}"
                    })
    except Exception as e:
        print(f"エラー（{img_name}）: {e}")

# CSV出力
df = pd.DataFrame(records)
output_path = base_dir / "ナラ枯れ_抽出セル.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"CSVを出力完了: {output_path}")

▶ 処理中: 八王子霊園
▶ 処理中: 小山内裏公園
▶ 処理中: 小山田緑地
▶ 処理中: 平山城址公園
▶ 処理中: 桜ヶ丘公園
▶ 処理中: 長沼公園
CSVを出力完了: c:\Users\kyous\OneDrive\デスクトップ\ハッカソン\tokyo-tree-doctor_program\tokyo-tree-doctor\ml\tests\EDA\ナラ枯れ_抽出セル.csv
